# Building complex widget libraries

## The problem
Often when composing widgets into an application, it becomes cumbersome to find a good way of managing state and keeping track of variables as the application grows. This notebooks aims to provide gentle guidance for managing complex application libraries. Please bear in mind these are merely **suggestions** and are by no means the only, or even best, way of going about this.

### The current approach

In [ ]:
# Generating data
import numpy as np
import pandas as pd

np.random.seed(0)
p_t, n = 100, 260
stock_df = pd.DataFrame({f'Stock {i}': p_t + np.round(np.random.standard_normal(n).cumsum(), 2) for i in range(10)})

In [ ]:
# Imports
from bqplot import LinearScale, Axis, Figure, Lines, CATEGORY10
from ipywidgets import HBox, VBox, Layout, HTML
from ipydatagrid import DataGrid
    
# Setting up the data grid
stock_grid = DataGrid(stock_df, selection_mode='column')

# Creating the bqplot chart objects
sc_x = LinearScale()
sc_y = LinearScale()
line = Lines(x=[], y=[], labels=['Fake stock price'], display_legend=True,
                 scales={'x': sc_x, 'y': sc_y})
ax_x = Axis(scale=sc_x, label='Index')
ax_y = Axis(scale=sc_y, orientation='vertical', label='y-value')
fig = Figure(marks=[line], axes=[ax_x, ax_y], title='Line Chart', layout=Layout(flex='1 1 auto', width='100%'))

# Creating the application title
app_title = HTML(value="<h1 style='color: salmon'>My complex application</h1><h2>Select a column to plot it</h2>")

In [ ]:
# Define callbacks
def plot_stock(*args):
    line.y = stock_grid.selected_cell_values
    line.x = range(len(line.y))
    column_index = stock_grid.selections[0]['c1']
    line.labels = [stock_df.columns[column_index]]
    line.colors = [CATEGORY10[np.random.randint(0, len(CATEGORY10)) % len(CATEGORY10)]]
    
# Event listener for cell click
stock_grid.observe(plot_stock, names='selections')

In [ ]:
VBox([
    app_title,
    HBox(
        [stock_grid, fig]
    )
])

### A more structured approach

In [ ]:
class Chart:
    def __init__(self, figure_title='Line Chart'):
        self._sc_x = LinearScale()
        self._sc_y = LinearScale()
        self._line = Lines(x=[], y=[], labels=['Fake stock price'], display_legend=True,
                         scales={'x': self._sc_x, 'y': self._sc_y})
        self._ax_x = Axis(scale=self._sc_x, label='Index')
        self._ax_y = Axis(scale=self._sc_y, orientation='vertical', label='y-value')
        self._fig = Figure(marks=[self._line], axes=[self._ax_x, self._ax_y], title=figure_title, layout=Layout(flex='1 1 auto', width='100%'))
        
    def get_figure(self):
        return self._fig
    
    def get_line(self):
        return self._line
    
    def set_line(self, x, y, labels, colors):
        self._line.x = x
        self._line.y = y
        self._line.labels = labels
        self._line.colors = colors

In [ ]:
from IPython.display import display

class MyApplication:
    def __init__(self, data, application_title='My complex application'):
        self.dataframe = data
        self.datagrid = self.process_data(data)
        self.chart = Chart()
        self.app_title = HTML(value=f"<h1 style='color: salmon'>{application_title}</h1><h2>Select a column to plot it</h2>")
        self.run_application()
        
    def process_data(self, dataframe):
        return DataGrid(dataframe, selection_mode='column')
        
    def generate_layout(self):
        return VBox([self.app_title, HBox([self.datagrid, self.chart.get_figure()])])
    
    def setup_event_handlers(self):
        self.datagrid.observe(self.plot_stock, names='selections')
    
    def run_application(self):
        self.setup_event_handlers()
        display(self.generate_layout())
        
    # Callbacks section
    def plot_stock(self, *args):
        column_index = self.datagrid.selections[0]['c1']
        line = self.chart.get_line()
        selected_values = self.datagrid.selected_cell_values
        self.chart.set_line(
            range(len(selected_values)), 
            selected_values, 
            [self.dataframe.columns[column_index]], 
            [CATEGORY10[np.random.randint(0, len(CATEGORY10)) % len(CATEGORY10)]]
        )

In [ ]:
app = MyApplication(data=stock_df, application_title="An alternative approach")